# 1. 국토부 오픈API 아파트 실거래가 수집

In [133]:
 ! pip install xmltodict  # 국토부 실거래 데이터 xml -> json 변환

In [134]:
import os
import pandas as pd
import folium
import json
import seaborn as sns
import xmltodict
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, quote
from folium.plugins import HeatMap

In [135]:
molit_serviceKey = '발급받은서비스키입력'

In [136]:
def molit_public_api(key, rcode, dealdate):
  try:
    print(rcode, dealdate)
    base = "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?"

    try:
      parameters = "serviceKey=%s&" % (key) + urlencode({quote_plus('LAWD_CD'): rcode, quote_plus('DEAL_YMD'): str(dealdate), quote_plus('numOfRows'): str(9999)}, encoding='UTF-8')
    except:
      return None
        
    url = base + parameters
    # print('url', url)

    request = Request(url)
    response = urlopen(request)
    rescode = response.getcode()
    # print(rescode)
    if rescode == 200:
      response_body = response.read().decode('utf-8')
    else:
      print('error code :' + rescode)

    if response_body == None:
      return None

    jsonData = json.loads(json.dumps(xmltodict.parse(response_body)))
    # print(jsonData['response']['body']['items']['item'])
    df = pd.DataFrame(jsonData['response']['body']['items']['item'])
    print(df.shape)
    return df

  except Exception as e:
    print('exception occur!', str(e))

In [137]:
def get_yearmonth_list(year, month):
    this_yearmonth = date.today()
    init_yearmonth = datetime.strptime(year+month, "%Y%m").date()
    year_months = []
    while init_yearmonth <= this_yearmonth:
        year_months.append(init_yearmonth)
        init_yearmonth = init_yearmonth + relativedelta(months=1)
    return year_months

In [138]:
year_months = get_yearmonth_list(year='2020', month='1')

In [139]:
for year_month in year_months:
    dirname = '/content/drive/MyDrive/국토부_실거래'
    yearmonth = year_month.strftime("%Y%m") #202102
    
    filename = os.path.join(dirname, '{}.csv'.format(yearmonth))
    if not os.path.isfile(filename):
      df_molit_api = molit_public_api(molit_serviceKey, rcode='36110', dealdate=yearmonth)
      df_molit_api.to_csv(filename, encoding='utf8', index=False)
    else:
      print(yearmonth, '이미 파일이 존재합니다.')

36110 202001
(1079, 13)
36110 202002
(1054, 13)
36110 202003
(463, 13)
36110 202004
(370, 13)
36110 202005
(665, 13)
36110 202006
(1543, 13)
36110 202007
(1392, 13)
36110 202008
(709, 13)
36110 202009
(318, 13)
36110 202010
(264, 13)
36110 202011
(602, 13)
36110 202012
(1154, 13)
202101 이미 파일이 존재합니다.
202102 이미 파일이 존재합니다.
202103 이미 파일이 존재합니다.
202104 이미 파일이 존재합니다.


In [140]:
df = pd.read_csv('/content/drive/MyDrive/국토부_실거래/202002.csv')
df.head()

,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,층,해제사유발생일,해제여부
0,"47,000",2019,2020,반곡동,수루배마을2단지,2,4,62.114,가-4101-2,36110,5,NaN,NaN
1,"80,000",2019,2020,반곡동,수루배마을1단지,2,5,96.788,가-4102-1,36110,20,NaN,NaN
2,"65,000",2019,2020,반곡동,수루배마을4단지(세종더샵예미지),2,8,84.890,가-4102-3,36110,10,NaN,NaN
3,"69,000",2019,2020,반곡동,수루배마을1단지,2,10,84.454,가-4102-1,36110,15,NaN,NaN
4,"50,000",2019,2020,반곡동,수루배마을4단지(세종더샵예미지),2,10,94.970,가-4102-3,36110,17,NaN,NaN


# 2. 국토부 오픈API 아파트 실거래가 데이터 불러오기

In [145]:
# start_y, end_y : YEAR 4자리
# start_m, end_m : MONTH 1~2자리
def get_file_list(start_y, start_m, end_y, end_m):
  if len(str(start_m)) == 1:
    start_m = '{0:02d}'.format(start_m)
  if len(str(end_m)) == 1:
    end_m = '{0:02d}'.format(end_m)

  start = int(str(start_y) + str(start_m))
  end = int(str(end_y) + str(end_m))

  print('조회시작:{}, 조회종료:{}'.format(start, end))

  temp_list = []
  sum = 0
  for file in os.walk('/content/drive/MyDrive/국토부_실거래/').__next__()[2]:
    fileName, fileExt = os.path.splitext(file) # 파일명과 파일확장자 구분
    if start <= int(fileName) <= end:
      df = pd.read_csv(os.path.join('/content/drive/MyDrive/국토부_실거래/', file), index_col=False)
      temp_list.append(df)
  
  df = pd.concat(temp_list, axis=0)
  print('조회완료!')
  return df

In [146]:
df = get_file_list(start_y=2020, start_m=1, end_y=2020, end_m=12)
df.head()

조회시작:202001, 조회종료:202012
조회완료!
